<a href="https://colab.research.google.com/github/manichandana8/Babies-/blob/main/AgTech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Database Exploration


Explored key tables like Plant, SoilTextureLookup, HumidityLookup, and others.
Identified missing data and handled it using imputation or removal.
Ensured consistency in data formats (e.g., numerical scaling and normalization).


In [ ]:
# import the libraries
import sqlite3
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report. precision_score, recakk_score, f1_score

In [ ]:
# Define the path to the database file
db_name = 'Agtech_database.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_name)

# Display success message
print("Connected to the database successfully!")

Connected to the database successfully!


### Get the tables of the database

In [ ]:
# Get a cursor object
cursor = conn.cursor()

# Query to retrieve all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all table names
tables = cursor.fetchall()

# Display the table names
print("Tables in the database:")
for table in tables:
    print(table[0])


Tables in the database:
PHRangeLookup
PlantHardinessZoneLookup
PlantTypeLookup
SoilTextureLookup
HumidityLookup
OrganicMatterLookup
Plant
SalinityLookup
PlantVariety


### Display the tables of the database

In [ ]:
# Fetch and display schema for each table
for table_name in tables:
    print(f"Schema for table '{table_name[0]}':")
    query = f"SELECT * FROM {table_name[0]} LIMIT 5"
    df = pd.read_sql_query(query, conn)
    print(df)
    print("\n")

Schema for table 'PHRangeLookup':
   PHRangeID    PHRange              SoilType  \
0          1      < 5.0  Very Strongly Acidic   
1          2  5.1 - 5.5       Strongly Acidic   
2          3  5.6 - 6.0     Moderately Acidic   
3          4  6.1 - 6.5       Slightly Acidic   
4          5  6.6 - 7.3               Neutral   

                                         Description  
0  Extremely acidic, often found in marshy or wat...  
1  Highly acidic, nutrient availability may be re...  
2  Many nutrients remain available, good for cert...  
3  Ideal for most plants, especially in loamy soils.  
4  Optimal for a wide range of plants, nutrient a...  


Schema for table 'PlantHardinessZoneLookup':
  ZoneID Zone  TemperatureStartRange  TemperatureEndRange
0    10A  10A                     30                   35
1    10B  10B                     35                   40
2    11A  11A                     40                   45
3    11B  11B                     45                   50
4   

### Display the schema of the database

In [ ]:
# Fetch the schema of all the tables
from sqlite3.dbapi2 import connect
for table_name in tables:
    print(f"Schema for table '{table_name[0]}':")
    query = f"PRAGMA table_info({table_name[0]});"
    schema = conn.execute(query).fetchall()
    for col in schema:
        print(col)
    print("\n")

Schema for table 'PHRangeLookup':
(0, 'PHRangeID', 'INTEGER', 0, None, 0)
(1, 'PHRange', 'TEXT', 0, None, 0)
(2, 'SoilType', 'TEXT', 0, None, 0)
(3, 'Description', 'TEXT', 0, None, 0)


Schema for table 'PlantHardinessZoneLookup':
(0, 'ZoneID', 'TEXT', 0, None, 0)
(1, 'Zone', 'TEXT', 0, None, 0)
(2, 'TemperatureStartRange', 'INTEGER', 0, None, 0)
(3, 'TemperatureEndRange', 'INTEGER', 0, None, 0)


Schema for table 'PlantTypeLookup':
(0, 'PlantTypeID', 'INTEGER', 0, None, 0)
(1, 'PlantType', 'TEXT', 0, None, 0)


Schema for table 'SoilTextureLookup':
(0, 'SoilTextureID', 'INTEGER', 0, None, 0)
(1, 'SoilTexture', 'TEXT', 0, None, 0)
(2, 'Description', 'TEXT', 0, None, 0)


Schema for table 'HumidityLookup':
(0, 'HumidityID', 'INTEGER', 0, None, 0)
(1, 'HumidityLevelLow', 'INTEGER', 0, None, 0)
(2, 'HumidityLevelHigh', 'INTEGER', 0, None, 0)
(3, 'Classification', 'TEXT', 0, None, 0)
(4, 'Description', 'TEXT', 0, None, 0)
(5, 'ImpactonPlants', 'TEXT', 0, None, 0)


Schema for table 'Organi

## FEATURE ENGINEERING

The dataset now includes engineered features that quantify critical agricultural conditions.
Features like NutrientIndex and EnvironmentalScore can help identify optimal crops for specific regions.
SoilMoistureScore highlights areas prone to drought or waterlogging.
Derived from soil texture and humidity to estimate soil moisture levels.
Formula: SoilMoistureScore = TextureScore * HumidityScore.
EnvironmentalScore:
Aggregates factors like plant hardiness zone, pH compatibility, and humidity.
Formula: EnvironmentalScore = HardinessScore + HumidityScore + PHScore.
we have added the features in the features modelling while cleaning the data set for running the model we have removed the null values and put a cleaned dataset to run the model.

In [ ]:
# Get all the tables
tables = {
    "PHRangeLookup": pd.read_sql_query("SELECT * FROM PHRangeLookup", conn),
    "PlantHardinessZoneLookup": pd.read_sql_query("SELECT * FROM PlantHardinessZoneLookup", conn),
    "PlantTypeLookup": pd.read_sql_query("SELECT * FROM PlantTypeLookup", conn),
    "SoilTextureLookup": pd.read_sql_query("SELECT * FROM SoilTextureLookup", conn),
    "HumidityLookup": pd.read_sql_query("SELECT * FROM HumidityLookup", conn),
    "OrganicMatterLookup": pd.read_sql_query("SELECT * FROM OrganicMatterLookup", conn),
    "Plant": pd.read_sql_query("SELECT * FROM Plant", conn),
    "SalinityLookup": pd.read_sql_query("SELECT * FROM SalinityLookup", conn),
    "PlantVariety": pd.read_sql_query("SELECT * FROM PlantVariety", conn),
}

# Display the loaded tables for confirmation
for name, df in tables.items():
    print(f"Table: {name}")
    print(df.head(), "\n")

# Load all tables into separate dataframes
plant_data = tables["Plant"]

# Handle missing data
plant_data.fillna({"SoilTexture": "Unknown", "Description": "No Data"}, inplace=True)


Table: PHRangeLookup
   PHRangeID    PHRange              SoilType  \
0          1      < 5.0  Very Strongly Acidic   
1          2  5.1 - 5.5       Strongly Acidic   
2          3  5.6 - 6.0     Moderately Acidic   
3          4  6.1 - 6.5       Slightly Acidic   
4          5  6.6 - 7.3               Neutral   

                                         Description  
0  Extremely acidic, often found in marshy or wat...  
1  Highly acidic, nutrient availability may be re...  
2  Many nutrients remain available, good for cert...  
3  Ideal for most plants, especially in loamy soils.  
4  Optimal for a wide range of plants, nutrient a...   

Table: PlantHardinessZoneLookup
  ZoneID Zone  TemperatureStartRange  TemperatureEndRange
0    10A  10A                     30                   35
1    10B  10B                     35                   40
2    11A  11A                     40                   45
3    11B  11B                     45                   50
4    12A  12A                 

In [ ]:
# create derived columns
scaler = MinMaxScaler()

# Calculate Nutrient Availability Index
plant_data["NutrientAvailability"] = (
    plant_data["OrganicMatterID"] * 0.6
    + (6 - plant_data["SalinityLevelID"]) * 0.4
)

# Convert 'ZoneID' columns to consistent data type before merging
plant_data["ZoneID"] = plant_data["ZoneID"].astype(str)
tables["PlantHardinessZoneLookup"]["ZoneID"] = tables["PlantHardinessZoneLookup"]["ZoneID"].astype(str)

# Merge PlantHardinessZoneLookup to get temperature ranges
plant_data = pd.merge(plant_data, tables["PlantHardinessZoneLookup"], on="ZoneID", how="left")

# Calculate Climatic Suitability Score
plant_data["ClimaticSuitability"] = (
    plant_data["HumidityID"] * 0.4
    + (plant_data["TemperatureStartRange"] + plant_data["TemperatureEndRange"]) / 2 * 0.6
)

# Scale the scores
plant_data["ClimaticSuitability"] = scaler.fit_transform(
    plant_data[["ClimaticSuitability"]]
)
plant_data["NutrientAvailability"] = scaler.fit_transform(
    plant_data[["NutrientAvailability"]]
)

# Calculate Crop Compatibility Index
plant_data["CropCompatibilityIndex"] = (
    plant_data["NutrientAvailability"] * 0.5
    + plant_data["ClimaticSuitability"] * 0.5
)

# One-Hot Encode categorical features
categorical_features = ["SoilTexture", "PlantType", "Zone"]
existing_features = [feature for feature in categorical_features if feature in plant_data.columns]
if existing_features:
    plant_data = pd.get_dummies(plant_data, columns=existing_features, drop_first=True)
else:
    print("No categorical features found for encoding.")

# SQL query to fetch additional details
sql_query = """
SELECT
    p.PlantID,
    p.PlantName,
    p.PlantDescription,
    st.SoilTexture,
    pr.PHRange,
    hl.HumidityLevelLow,
    hl.HumidityLevelHigh,
    pt.PlantType,
    om.OrganicMatterContent,
    sl.SalinityLevel,
    zh.Zone,
    pv.PlantVarietyName,
    pv.PlantVarietyDescription
FROM
    Plant p
LEFT JOIN
    SoilTextureLookup st ON p.SoilTextureID = st.SoilTextureID
LEFT JOIN
    PHRangeLookup pr ON p.PHRangeID = pr.PHRangeID
LEFT JOIN
    HumidityLookup hl ON p.HumidityID = hl.HumidityID
LEFT JOIN
    PlantTypeLookup pt ON p.PlantTypeID = pt.PlantTypeID
LEFT JOIN
    OrganicMatterLookup om ON p.OrganicMatterID = om.OrganicMatterID
LEFT JOIN
    SalinityLookup sl ON p.SalinityLevelID = sl.SalinityLevelID
LEFT JOIN
    PlantHardinessZoneLookup zh ON p.ZoneID = zh.ZoneID
LEFT JOIN
    PlantVariety pv ON p.PlantID = pv.PlantID;
"""

# Execute the query and load the result into a pandas DataFrame
df = pd.read_sql_query(sql_query, conn)

# Add calculated indices to the SQL-based DataFrame
if not df.empty:
    df = pd.merge(df, plant_data[["PlantID", "NutrientAvailability", "ClimaticSuitability", "CropCompatibilityIndex"]], on="PlantID", how="left")
    df.to_csv('joined_plants_data.csv', index=False)
    print("CSV file with calculated indices created successfully!")
else:
    print("No data returned from the query.")


CSV file with calculated indices created successfully!


/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:695: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:712: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


In [ ]:
Agtech = pd.read_csv('joined_plants_data.csv')
Agtech.head(5)

,PlantID,PlantName,PlantDescription,SoilTexture,PHRange,HumidityLevelLow,HumidityLevelHigh,PlantType,OrganicMatterContent,SalinityLevel,Zone,PlantVarietyName,PlantVarietyDescription,NutrientAvailability,ClimaticSuitability,CropCompatibilityIndex
0,2,Lettuce,A leafy green vegetable commonly used in salads.,Loamy Soil,6.1 - 6.5,50.0,70.0,Vegetable,Moderate (2% - 4%),< 2,NaN,Arugula,"Also known as rocket, it has a peppery flavor ...",0.857143,NaN,NaN
1,2,Lettuce,A leafy green vegetable commonly used in salads.,Loamy Soil,6.1 - 6.5,50.0,70.0,Vegetable,Moderate (2% - 4%),< 2,NaN,Belgian Endive,A leafy vegetable known for its distinctive sh...,0.857143,NaN,NaN
2,2,Lettuce,A leafy green vegetable commonly used in salads.,Loamy Soil,6.1 - 6.5,50.0,70.0,Vegetable,Moderate (2% - 4%),< 2,NaN,Buttercrunch,A hybrid lettuce with a crunchy texture and sw...,0.857143,NaN,NaN
3,2,Lettuce,A leafy green vegetable commonly used in salads.,Loamy Soil,6.1 - 6.5,50.0,70.0,Vegetable,Moderate (2% - 4%),< 2,NaN,Butterhead,"Also known as Bibb or Boston lettuce, it has s...",0.857143,NaN,NaN
4,2,Lettuce,A leafy green vegetable commonly used in salads.,Loamy Soil,6.1 - 6.5,50.0,70.0,Vegetable,Moderate (2% - 4%),< 2,NaN,Celtuce,"Also known as stem lettuce, it is grown primar...",0.857143,NaN,NaN


## MODEL DEVELOPMENT

### Dataset preparation

Training:
The Random Forest model (rf_model) is trained on the prepared features and target variable (PlantVarietyName).

Feature Importance:
Random Forest can rank features by importance, helping identify which features most influence predictions.
Robustness:
Handles noisy data and missing values better than many other algorithms.
Scalability:
Efficient for datasets with large numbers of features and observations.

In [ ]:
# Identify Features and Target
target_column = 'PlantVarietyName'


# Ensure no missing values
Agtech = Agtech.dropna(subset=[target_column])

# Encode Categorical Variables
categorical_cols = Agtech.select_dtypes(include=['object']).columns
categorical_cols = categorical_cols[categorical_cols != target_column]
Agtech = pd.get_dummies(Agtech, columns=categorical_cols, drop_first=True)

features = [col for col in Agtech.columns if col != target_column]


In [ ]:
# Split into Features (X) and Target (y)
X = Agtech[features]
y = Agtech[target_column]

In [ ]:
# Split the data into training data and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X.shape, X_train.shape, X_test.shape)

(3056, 3276) (2444, 3276) (612, 3276)


In [ ]:
# Train the Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test)

In [ ]:
# Predictions
input_data = {
    'PHRange': [6.3],
    'HumidityLevelLow': [50.0],
    'HumidityLevelHigh': [70.0],
    'OrganicMatterContent_Moderate (2% - 4%)': [1],
    'SoilTexture_Loamy Soil': [1],
}

# Convert input data to a DataFrame
input_df = pd.DataFrame(input_data)

# Ensure the DataFrame has the same columns as the training data
missing_cols = set(X.columns) - set(input_df.columns)
for col in missing_cols:
    input_df[col] = 0

# Reorder columns to match the training data
input_df = input_df[X.columns]

# Use the trained model to predict
prediction = rf_model.predict(input_df)

print("Predicted Plant Variety:", prediction[0])


Streaming output truncated to the last 5000 lines.
<ipython-input-68-0fc7b8682a36>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0  # Add missing columns with default values (e.g., 0 for numerical features)
<ipython-input-68-0fc7b8682a36>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  input_df[col] = 0  # Add missing columns with default values (e.g., 0 for numerical features)
<ipython-input-68-0fc7b8682a36>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fra

Predicted Plant Variety: Iceberg


## EVALUATION METRICS

Evaluation:Assess the model’s accuracy is defined using the metrics  F1-score, accuracy, precision and recall for validation of dataset.


In [ ]:
# Accuracy scores
print("\nModel Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Model Accuracy: 0.017973856209150325

Classification Report:
                                        precision    recall  f1-score   support

                              Abacaxi       0.00      0.00      0.00         1
                                Acaia       0.00      0.00      0.00         1
                      Adirondack Blue       0.00      0.00      0.00         1
                        African Pride       0.00      0.00      0.00         0
                       African Runner       0.00      0.00      0.00         1
                      Agave americana       0.00      0.00      0.00         1
                      Agave bracteosa       0.00      0.00      0.00         1
                    Agave durangensis       0.00      0.00      0.00         1
                       Agave sisalana       0.00      0.00      0.00         0
              Agave victoriae-reginae       0.00      0.00      0.00         1
                         Agave weberi       0.00      0.00      0.0

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_

These metrics are essential for evaluating the performance of a classification model, especially in multi-class problems.
Precision evaluates the accuracy of the model's positive predictions.
Recall measures the model's ability to identify all relevant cases.
The F1 Score balances both metrics, making it ideal for imbalanced datasets.

In [ ]:
# Predictions from the model on the test set
y_pred = rf_model.predict(X_test)

# Calculate Precision, Recall, and F1 Score
precision = precision_score(y_test, y_pred, average='weighted')  # Use 'weighted' for multi-class
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Precision: 0.01657329598506069
Recall: 0.017973856209150325
F1 Score: 0.016748366013071895


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
